# CNN from scratch - Build a Custom Conv2d Module

In [1]:
# show version and all available gpu devices
import torch
print(torch.__version__)
if torch.cuda.is_available():
    for idx in range(torch.cuda.device_count()):
        print(torch.cuda.get_device_name(idx))
else:
    print('No GPU available')

2.5.1
NVIDIA GeForce GTX 1070 Ti


## Implementing

MyConv2d is a custom re-implementation of the CNN. 

2 versions were implemented. You can test these by commenting/un-commenting the code in the forward function :
- `self.my_forward(x)` : A very slow but readable pythonic way of computing the convolutions
- `self.fold_forward(x)` : Take advantages of methods from the Torch package like folding/unfolding to compute a forward pass in a better optimized way. (still slower than default as it runs in the interpreter as opposed to the C++ compiled modules) 

Which you can compare to the default method `self.default_forward`.

In [2]:
from torch import nn


class MyConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding='same', dilation=1, groups=1, bias=True):
        super(MyConv2d, self).__init__()        
        self.stride = (stride, stride) if isinstance(stride, int) else tuple(stride)
        self.dilation = (dilation, dilation) if isinstance(dilation, int) else tuple(dilation)
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else tuple(kernel_size)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.groups = self.check_params(groups)
        self.bias = bias

        if isinstance(padding, str):
            if padding.lower() == 'same':
                self.padding = tuple((s * (k - 1) // 2 for s, k in zip(self.stride, self.kernel_size)))
            elif padding.lower() == 'valid':
                self.padding = (0, 0)
            else:
                raise ValueError('Padding must be "same", "valid", or an integer.')
        else:
            self.padding = (padding, padding) if isinstance(padding, int) else tuple(padding)

        self.weight = nn.Parameter(
            torch.randn(out_channels, in_channels // groups, kernel_size, kernel_size)
        )
        self.bias = nn.Parameter(torch.randn(out_channels)) if bias else None
        
    def forward(self, x):
        # return self.my_forward(x) # a lot slower since it's not optimized, doesn't exploit parallelism
        return self.fold_forward(x)  # Optimized version using matrix multiplication and unfolding with parallelism
        # return self.default_forward(x) # Default PyTorch implementation
    
    def my_forward(self, x):
        """Reproduces F.conv2d with the specified parameters in a pythonic way without the explicit call to the function."""
        batch_size, in_channels, height, width = x.size()

        # Check dimensions
        assert in_channels == self.in_channels, \
            f'Expected input with {self.in_channels} channels, but got {in_channels} channels'

        # Calculate output dimensions
        out_height = (height + 2 * self.padding[0] - self.dilation[0] * (self.kernel_size[0] - 1) - 1) // self.stride[0] + 1
        out_width = (width + 2 * self.padding[1] - self.dilation[1] * (self.kernel_size[1] - 1) - 1) // self.stride[1] + 1

        # Apply padding if needed
        if self.padding != (0, 0):
            x = nn.functional.pad(x, (self.padding[1], self.padding[1], self.padding[0], self.padding[0]))

        # Initialize the output tensor
        out = torch.zeros((batch_size, self.out_channels, out_height, out_width), device=x.device)

        # Perform the convolution operation
        out_channels_per_group = self.out_channels // self.groups
        in_channels_per_group = self.in_channels // self.groups

        for g in range(self.groups):
            for i in range(out_height):
                for j in range(out_width):
                    h_start = i * self.stride[0]
                    h_end = h_start + self.kernel_size[0] * self.dilation[0]
                    w_start = j * self.stride[1]
                    w_end = w_start + self.kernel_size[1] * self.dilation[1]

                    x_slice = x[:, g * in_channels_per_group:(g + 1) * in_channels_per_group, h_start:h_end:self.dilation[0], w_start:w_end:self.dilation[1]]

                    out[:, g * out_channels_per_group:(g + 1) * out_channels_per_group, i, j] = \
                        torch.einsum('bijk,oijk->bo', x_slice, self.weight[g * out_channels_per_group:(g + 1) * out_channels_per_group]) + \
                        self.bias[g * out_channels_per_group:(g + 1) * out_channels_per_group]

        return out

    def fold_forward(self, x):
        """Reproduces F.conv2d with the specified parameters using torch methods"""
        n_batch, in_channels, in_height, in_width = x.size()
        out_height = (in_height + 2 * self.padding[0] - self.dilation[0] * (self.kernel_size[0] - 1) - 1) // self.stride[0] + 1
        out_width = (in_width + 2 * self.padding[1] - self.dilation[1] * (self.kernel_size[1] - 1) - 1) // self.stride[1] + 1
    
        # Pad the input tensor
        if self.padding != (0, 0):
            x = nn.functional.pad(x, (self.padding[1], self.padding[1], self.padding[0], self.padding[0]))
    
        # Unfold the input tensor to apply the convolution operation
        x = torch.nn.functional.unfold(x, self.kernel_size, dilation=self.dilation, stride=self.stride)
    
        # Reshape the weights
        weight = self.weight.view(self.weight.size(0), -1)
    
        # Perform matrix multiplication
        x = torch.matmul(x.permute(0, 2, 1), weight.t())
        
        # Reshape the output tensor
        x = x.view(n_batch, self.out_channels, out_height, out_width)
    
        return x + self.bias.view(1, -1, 1, 1) if self.bias is not None else x


    def default_forward(self, x):
        return F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)

    def check_params(self, groups):
        """Checks the parameters of the Conv2d layer to ensure they are valid."""
        if groups <= 0:
            raise ValueError('groups must be a positive integer')
        if self.in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if self.out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        return groups

## Testing

Below is some code to test on MNIST. BEWARE, the groups and dilation parameters were not verified, only the kernel_size, stride, and padding.

In [3]:
import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
import torchmetrics

class MNISTModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.mnist_val = None
        self.mnist_train = None
        self.mnist_test = None
        self.convBlock1 = nn.Sequential(
            MyConv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.convBlock2 = nn.Sequential(
            MyConv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_out = nn.Linear(64 * 7 * 7, 10)
        
        # Initialize metrics
        self.train_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=10)
        self.val_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=10)
        self.test_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=10)

    def forward(self, x):
        x = self.convBlock1(x)
        x = self.convBlock2(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        return self.fc_out(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        
        # Calculate and log accuracy
        preds = torch.argmax(logits, dim=1)
        acc = self.train_accuracy(preds, y)
        self.log('train_loss', loss)
        self.log('train_acc', acc, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        
        # Calculate and log accuracy
        preds = torch.argmax(logits, dim=1)
        acc = self.val_accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        
        # Calculate and log accuracy
        preds = torch.argmax(logits, dim=1)
        acc = self.test_accuracy(preds, y)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def prepare_data(self):
        # Download and prepare the MNIST dataset
        datasets.MNIST('../data/raw', train=True, download=True)
        datasets.MNIST('../data/raw', train=False, download=True)

    def setup(self, stage=None):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
        if stage == 'fit' or stage is None:
            mnist_full = datasets.MNIST('../data/raw', train=True, transform=transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])
        if stage == 'test' or stage is None:
            self.mnist_test = datasets.MNIST('../data/raw', train=False, transform=transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=256, num_workers=8, shuffle=True, persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=128, num_workers=8, shuffle=False, persistent_workers=True)
    
    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=128, num_workers=8, shuffle=False, persistent_workers=True)

In [4]:
# EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss', # Metric to monitor
    patience=3,         # Number of epochs with no improvement after which training will be stopped
    verbose=True,       # Print messages when early stopping is triggered
    mode='min'          # Minimize the monitored metric
)

# Trainer with EarlyStopping callback
trainer = Trainer(max_epochs=50, callbacks=[early_stopping], accelerator='gpu', devices=1, default_root_dir='../models/cnn-scratch')
model = MNISTModel()
trainer.fit(model)

# Get best weights from checkpoint
best_model_path = trainer.checkpoint_callback.best_model_path
model = MNISTModel.load_from_checkpoint(best_model_path)

# Test the model
trainer.test(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:01<00:00, 8.61MB/s]


Extracting ../data/raw/MNIST/raw/train-images-idx3-ubyte.gz to ../data/raw/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 329kB/s]


...

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.348. Signaling Trainer to stop.


Epoch 17: 100%|██████████| 215/215 [00:05<00:00, 41.00it/s, v_num=0, train_acc=0.995, val_loss=0.354, val_acc=0.973]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 79/79 [00:00<00:00, 131.62it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9754999876022339
        test_loss           0.3307338058948517
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.3307338058948517, 'test_acc': 0.9754999876022339}]

In [6]:
%reload_ext tensorboard
%tensorboard --logdir=../models/cnn-scratch/lightning_logs/

Reusing TensorBoard on port 6006 (pid 6812), started 0:00:13 ago. (Use '!kill 6812' to kill it.)